To note:

- No idea what b is 
- Below is a couple of definitions from the GitHub page which might come in handy

Link travel time = free flow time * ( 1 + B * (flow/capacity)^Power ).
Link generalized cost = Link travel time + toll_factor * toll + distance_factor * distance

# Setup: import packages

In [131]:
#install packages if required
#%pip install networkx

In [2]:
#import packages 
import os
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import graphviz
import scipy.integrate as integrate 
from scipy.optimize import minimize_scalar
from scipy.misc import derivative
from random import *

In [3]:
network_analysis_path_ben = '/Users/benseimon/Documents/Barca GSE/Studies/Term 2/Networks/Term Paper/Networks_Term_Paper/pytrans_UrbanNetworkAnalysis/pytrans/UrbanNetworkAnalysis/Editing Classes'
#network_analysis_path_niamh = r'C:\Users\35387\OneDrive\Documents\Networks\New folder\Networks_Term_Paper\pytrans_UrbanNetworkAnalysis\pytrans\UrbanNetworkAnalysis\original_edited'
os.chdir(network_analysis_path_ben)
import Frank_Wolfe
import TransportationNetworks as tn
import visualize_graph

# Read in data

data_path = '/Users/benseimon/Documents/Barca GSE/Studies/Term 2/Networks/Term Paper/Networks_Term_Paper/Data/'
city = 'Birmingham'
data_path = data_path + city + '/'
os.chdir(data_path)
link_file_path = data_path + city + '_net.tntp'
node_file_path = data_path + city +'_nodes.tntp'
trip_file_path = data_path+ city + '_trips.tntp'

# Load Data

In [4]:
data_path = '/Users/benseimon/Documents/Barca GSE/Studies/Term 2/Networks/Term Paper/Networks_Term_Paper/Data/'
cities = ['SiouxFalls']
data_path = data_path 
os.chdir(data_path)

In [5]:
#create dictionary with cities and three files: links, nodes and trips
#i apologise, the paths are horrible, you might need to fix but it's worth it in the long u
def make_dict(cities):
    cities_dict = {}
    for city in cities:
        link_file_path = data_path + city + '/' + city + '_net.tntp'
        node_file_path = data_path + city + '/' + city +'_nodes.tntp'
        trip_file_path = data_path + city + '/' + city + '_trips.tntp'
        cities_dict[city] = {}
        to_add = {'link_file_path': link_file_path, 'node_file_path': node_file_path, 'trip_file_path': trip_file_path}
        cities_dict[city]['file_paths'] = to_add 
    return cities_dict

In [6]:
cities_dict = make_dict(cities)
cities_dict

{'SiouxFalls': {'file_paths': {'link_file_path': '/Users/benseimon/Documents/Barca GSE/Studies/Term 2/Networks/Term Paper/Networks_Term_Paper/Data/SiouxFalls/SiouxFalls_net.tntp',
   'node_file_path': '/Users/benseimon/Documents/Barca GSE/Studies/Term 2/Networks/Term Paper/Networks_Term_Paper/Data/SiouxFalls/SiouxFalls_nodes.tntp',
   'trip_file_path': '/Users/benseimon/Documents/Barca GSE/Studies/Term 2/Networks/Term Paper/Networks_Term_Paper/Data/SiouxFalls/SiouxFalls_trips.tntp'}}}

In [7]:
class fw_custom_algorithm():
    
    def __init__(self, cities_dict):
        self.cities_dict = cities_dict
        self.link_fields = {"from": 1, "to": 2, "capacity": 3, "length": 4, "t0": 5, "B": 6, "beta": 7, "V": 8}
    #create a csv for each city and corresponding file for investigation
    def create_csvs(self):
        for city in self.cities_dict.keys():
            path_list = list(self.cities_dict[city]['file_paths'].keys())
            for paths in path_list:
                #load network file
                link_file = path_list[0]
                csv = pd.read_csv(self.cities_dict[city]['file_paths'][link_file], skiprows=8, sep='\t') #gets link file path
                trimmed= [s.strip().lower() for s in csv.columns]
                csv.columns = trimmed
                # And drop the silly first andlast columns
                csv.drop(['~', ';'], axis=1, inplace=True)
                cities_dict[city]['csv'] = csv
        self.cities_dict = cities_dict
    
    def plot(self, city):
        csv = self.cities_dict[city]['csv']
        network = nx.from_pandas_edgelist(csv, source = 'init_node',target='term_node', edge_attr = True)
        graph = nx.complete_graph(network)
        nx.draw(graph)
        
    def summary_stats_all(self):
        for city in self.cities_dict.keys():
            print('Total number of nodes in', city+':', len(self.cities_dict[city]['csv']['init_node'].unique()))
            print('Total number of edges in', city+':', self.cities_dict[city]['csv'].shape[0])
            
    def network(self, city):
        #instatiate class using required files 
        link_file = self.cities_dict[city]['file_paths']['link_file_path']
        trip_file = self.cities_dict[city]['file_paths']['trip_file_path']
        node_file = self.cities_dict[city]['file_paths']['node_file_path']
        Network = tn.Network(link_file, trip_file, node_file)
        ##Network has three attributes
        #1) graph object
        graph = Network.graph
        #2) origin nodes
        origin_nodes = Network.origins
        #3)dict: keys= origin_node, destination_node, value = traffic flow
        flows = Network.od_vols
        print(Network)
        print(city, 'graph is:', graph)
        print(city, 'origin nodes are:', origin_nodes)
        print(city, 'flows are:', flows)
        
    #define a function for finding the equilibrim flow using the Frank-Wolfe algorithm from PyTrans
    def compute_all_link_flow(self, city):
        link_file = self.cities_dict[city]['file_paths']['link_file_path']
        trip_file = self.cities_dict[city]['file_paths']['trip_file_path']
        node_file = self.cities_dict[city]['file_paths']['node_file_path']    
        SO = False
        fw = Frank_Wolfe.Run(link_file, trip_file, node_file, SO)
        #saves file to dict
        self.cities_dict[city]['all_links'] = {'all_links': fw}
    
    def compute_missing_link_flow(self, city):   
        link_file = self.cities_dict[city]['file_paths']['link_file_path']
        trip_file = self.cities_dict[city]['file_paths']['trip_file_path']
        node_file = self.cities_dict[city]['file_paths']['node_file_path'] 
        link = 0
        total_links = cities_dict[city]['csv'].shape[0]
        for link in list(range(0, total_links+1)):
            link_file_missing_link = self.remove_link(city, link_file, link)
            while link <= total_links:
                SO = False
                fw = Frank_Wolfe.Run(link_file_missing_link, trip_file, node_file, SO)
                self.cities_dict[city]['closed_links'] = {str(link) : fw}
                link += 1
    
    def remove_link(self, city, link_file, link):
        f = open(link_file)
        lines = f.readlines()
        f.close()
        
        
    
    

In [8]:
link_file = cities_dict['SiouxFalls']['file_paths']['link_file_path']

In [9]:
f = open(link_file)
lines = f.readlines()
f.close()

In [10]:
with open('your_file.txt', 'w') as f:
    for item in my_list:
        f.write("%s\n" % item)

NameError: name 'my_list' is not defined

In [11]:
link = 0
with open(link_file, "r") as f:
    lines = f.readlines()
    print(lines)
del lines[0]
with open(link_file, "w") as f:
    for line in lines:
        if line != link:
            f.write(line)
        else:
            pass

thefile.write('\n'.join(thelist))
f = open(link_file)
lines = f.readlines()
f.close()
print(lines)

['<NUMBER OF ZONES> 24\t\t\t\t\t\t\t\t\t\t\t\n', '<NUMBER OF NODES> 24\t\t\t\t\t\t\t\t\t\t\t\n', '<FIRST THRU NODE> 1\t\t\t\t\t\t\t\t\t\t\t\n', '<NUMBER OF LINKS> 76\t\n', '<ORIGINAL HEADER>~ \tInit node \tTerm node \tCapacity \tLength \tFree Flow Time \tB\tPower\tSpeed limit \tToll \tType\t;\n', '<END OF METADATA>\t\t\t\t\t\t\t\t\t\t\t\n', '\n', '\n', '~\tinit_node\tterm_node\tcapacity\tlength\tfree_flow_time\tb\tpower\tspeed\ttoll\tlink_type\t;\n', '\t1\t2\t25900.20064\t6\t6\t0.15\t4\t0\t0\t1\t;\n', '\t1\t3\t23403.47319\t4\t4\t0.15\t4\t0\t0\t1\t;\n', '\t2\t1\t25900.20064\t6\t6\t0.15\t4\t0\t0\t1\t;\n', '\t2\t6\t4958.180928\t5\t5\t0.15\t4\t0\t0\t1\t;\n', '\t3\t1\t23403.47319\t4\t4\t0.15\t4\t0\t0\t1\t;\n', '\t3\t4\t17110.52372\t4\t4\t0.15\t4\t0\t0\t1\t;\n', '\t3\t12\t23403.47319\t4\t4\t0.15\t4\t0\t0\t1\t;\n', '\t4\t3\t17110.52372\t4\t4\t0.15\t4\t0\t0\t1\t;\n', '\t4\t5\t17782.7941\t2\t2\t0.15\t4\t0\t0\t1\t;\n', '\t4\t11\t4908.82673\t6\t6\t0.15\t4\t0\t0\t1\t;\n', '\t5\t4\t17782.7941\t2\t2

NameError: name 'thefile' is not defined

In [12]:
open(link_file, "r")

<_io.TextIOWrapper name='/Users/benseimon/Documents/Barca GSE/Studies/Term 2/Networks/Term Paper/Networks_Term_Paper/Data/SiouxFalls/SiouxFalls_net.tntp' mode='r' encoding='UTF-8'>

In [13]:
#instantiate class 
trial_run = fw_custom_algorithm(cities_dict)

In [14]:
trial_run.summary_stats_all()

KeyError: 'csv'

#load node file
        brum_nodes = pd.read_csv(node_file_path, sep = ' ')
for_drop = []
for i in list(range(1,14)):
    name = 'Unnamed: ' + str(i)
    for_drop.append(name)
for_drop.remove('Unnamed: 7')
brum_nodes = brum_nodes.drop(for_drop, axis = 1)

#load trips file
#load node file - note this is a bit fiddly, could be a better way to import but chose something quick and dirty
brum_nodes = pd.read_csv(node_file_path, sep = ' ')
for_drop = []
for i in list(range(1,14)):
    name = 'Unnamed: ' + str(i)
    for_drop.append(name)
for_drop.remove('Unnamed: 7')
brum_nodes = brum_nodes.drop(for_drop, axis = 1)

# Plot graph

In [15]:
trial_run.create_csvs()
trial_run.plot('SiouxFalls')

KeyError: "['~'] not found in axis"

# Observe example network data

In [16]:
network(cities_dict, city)

NameError: name 'network' is not defined

# Compute equilibrium flow using Frank-Wolfe algorithm

In [17]:
for city in cities_dict.keys():
    trial_run.compute_all_link_flow(city)

TypeError: __init__() missing 3 required positional arguments: 'Node', 'Link', and 'Network'

In [18]:
trial_run.cities_dict['SiouxFalls']['all_links']['all_links'].showODFlow()
trial_run.cities_dict['SiouxFalls']['all_links']['all_links'].showODFlowMap()

KeyError: 'all_links'

In [19]:
trial_run.cities_dict['SiouxFalls']['csv']

KeyError: 'csv'

In [20]:
trial_run.cities_dict['SiouxFalls']['all_links']['all_links'].graph.edges

KeyError: 'all_links'

# Systematically close each link and compute equilibrium flow

In [21]:
for city in cities_dict.keys():
    train_run.compute_missing_link_flow(city)

NameError: name 'train_run' is not defined

In [22]:
trial_run.cities_dict['SiouxFalls']['csv']

KeyError: 'csv'